# Tabular Examples
RNA3DB contains `Tabular`, a high-level API for interacting with Infernal's output tables for `cmscan`.

Using the `.tbl` files we provide, we can look up Rfam family information for any RNA chain in the PDB.

In [1]:
# read an entire directory of *.tbl files, such as those provided with the release of RNA3DB
from rna3db.tabular import read_tbls_from_dir
tbl = read_tbls_from_dir('../data/cmscans/')

In [2]:
# we can query all the hits for any chain and get a formatted table
tbl['7pkt_7']

target_name         target_accession    query_name          score               e_value             
----------------------------------------------------------------------------------------------------
LSU_rRNA_bacteria   RF02541             7pkt_7              333.9               2.7e-109            
LSU_rRNA_archaea    RF02540             7pkt_7              225.7               1.8e-69             
LSU_rRNA_eukarya    RF02543             7pkt_7              151.5               6.9e-39             
TMV_UPD-PK3         RF01072             7pkt_7              14.5                0.67                
DENV_SLA            RF02340             7pkt_7              16.5                3.3                 

These hits are sorted by E-value, and contain all hits for a chain. 

All queries return a `Tabular` object, which we can do further queries on. For example, we can also filter these hits by E-value.

In [3]:
# filter hits by E-value
tbl['7pkt_7'].filter_e_value(1e-20)

target_name         target_accession    query_name          score               e_value             
----------------------------------------------------------------------------------------------------
LSU_rRNA_bacteria   RF02541             7pkt_7              333.9               2.7e-109            
LSU_rRNA_archaea    RF02540             7pkt_7              225.7               1.8e-69             
LSU_rRNA_eukarya    RF02543             7pkt_7              151.5               6.9e-39             

`Tabular` objects are also iterators comprised of `Hit`s. A singular `Hit` has the same attributes as Infernal hits table format 1, with some minor modifications to naming:
- `target_name`
- `target_accession`
- `query_name`
- `query_accession`
- `mdl`
- `mdl_from`
- `mdl_to`
- `seq_from`
- `seq_to`
- `strand`
- `trunc`
- `pass_n`
- `gc`
- `bias`
- `score`
- `e_value`
- `inc`
- `description_of_target`

For more information about these attributes, see the [Infernal Userguide](http://eddylab.org/infernal/Userguide.pdf).


In [4]:
# we can use Tabular objects as iterators
for hit in tbl['7pkt_7']:
    # we can access a Hit's attributes via dot notation
    print(hit.query_name, hit.description_of_target)

7pkt_7 Bacterial large subunit ribosomal RNA
7pkt_7 Archaeal large subunit ribosomal RNA
7pkt_7 Eukaryotic large subunit ribosomal RNA
7pkt_7 Pseudoknot of upstream pseudoknot domain (UPD) of the 3'UTR
7pkt_7 Dengue virus SLA


These attributes can also be accessed in `Tabular` objects with dot notation, which will then return a list of that attribute from the table.

This has various uses, such as getting all the Rfam accessions of a query, for example.

In [5]:
# we can chain queries on Tabular objects
tbl['7pkt_7'].filter_e_value(1e-3).target_accession

['RF02541', 'RF02540', 'RF02543']

We can also find the best hit for every chain in the query with `top_hits`. This will take the lowerst E-value hit for every chain.

For example, we can get the top hit for every chain that gets at least one hit with E-value < `1e-3`.

In [6]:
# get the top hits for each chain that matches the query
tbl.filter_e_value(1e-3).top_hits

target_name         target_accession    query_name          score               e_value             
----------------------------------------------------------------------------------------------------
SSU_rRNA_bacteria   RF00177             5uyq_A              1564.7              0.0                 
LSU_rRNA_bacteria   RF02541             5uyq_01             2879.9              0.0                 
SSU_rRNA_bacteria   RF00177             3jbv_A              1577.5              0.0                 
LSU_rRNA_bacteria   RF02541             3jbv_b              2889.8              0.0                 
SSU_rRNA_bacteria   RF00177             8cf8_A              1565.8              0.0                 
SSU_rRNA_bacteria   RF00177             5no2_A              1558.3              0.0                 
LSU_rRNA_archaea    RF02540             3ow2_0              2776.4              0.0                 
LSU_rRNA_bacteria   RF02541             7o5b_X              2986.5              0.0        

`Tabular` objects support the `len` function.

In [7]:
# get number of chains that have a hit with E-value lower than 1e-3
len(tbl.filter_e_value(1e-3).top_hits)

10380

It is also possible to query any attribute via the `filter_attr_by_set` and `filter_attr_by_value` methods.

You may wish to, for example, get hits for all the tRNA (`RF00005`) Rfam family.

In [8]:
# we can filter by any attribute
tbl.filter_attr_by_value('target_accession', 'RF00005')

target_name         target_accession    query_name          score               e_value             
----------------------------------------------------------------------------------------------------
tRNA                RF00005             5uyq_Y              78.5                8.4e-19             
tRNA                RF00005             5e7k_3K             78.5                8.4e-19             
tRNA                RF00005             5e7k_1L             78.5                8.4e-19             
tRNA                RF00005             5e7k_3L             78.5                8.4e-19             
tRNA                RF00005             5el4_1K             78.5                8.4e-19             
tRNA                RF00005             5el4_3K             78.5                8.4e-19             
tRNA                RF00005             5el4_1L             78.5                8.4e-19             
tRNA                RF00005             5el4_3L             78.5                8.4e-19    

The `filter_attr_by_set` method does the same thing as `filter_attr_by_value`, but we can provide a list rather than a single value.

This can be useful in many cases, such as trying to get hits for all tRNA clans.

In [9]:
# all Rfam accessions of tRNA clan
trna_clan = ['RF00005', 'RF00023', 'RF01849', 'RF01850', 'RF01851', 'RF01852', 'RF02544']
tbl.filter_attr_by_set('target_accession', trna_clan)

target_name         target_accession    query_name          score               e_value             
----------------------------------------------------------------------------------------------------
tmRNA               RF00023             6q98_4              231.6               1.1e-68             
tmRNA               RF00023             7ac7_4              231.6               1.1e-68             
tmRNA               RF00023             6q9a_4              231.6               1.1e-68             
tmRNA               RF00023             7acj_4              231.6               1.1e-68             
tmRNA               RF00023             4v6t_AV             231.6               1.1e-68             
tmRNA               RF00023             7abz_4              231.6               1.1e-68             
tmRNA               RF00023             7acr_4              231.6               1.1e-68             
tmRNA               RF00023             6q97_4              229.4               5.1e-68    

Putting these together, we can make some more complex queries.

For example, if we wanted to find out how many chains have a hit to the tRNA clan with an E-value lower than `1e-3`, we can do:

In [10]:
# filter to the tRNA clan, to 1e-3 E-value, get a list of query_names, make it a set, get length
len(set(tbl.filter_attr_by_set('target_accession', trna_clan).filter_e_value(1e-3).query_name))

2511